# Portfolio

In [1]:
import requests
import json
import urllib.request

In [114]:
import portfolio as p

In [62]:
s = p.get_portfolio()

In [64]:
def add_token(s, token, coingecko, network, address, decimals, asset=True):
    """
    - s is portfolio.json
    - token: RPL
    - coingecko: rocket-pool
    - network: arbitrum
    - address: 0x...
    - asset if asset, else debt
    """
    token_type = "asset" if asset else "debt"
    token = token.lower()
    if token not in s[token_type].keys():
        s[token_type][token] = {}
    s[token_type][token]["coingecko"] = coingecko
    s[token_type][token][network] = address
    s[token_type][token]["decimals"] = decimals

In [65]:
add_token(s, "rpl", "rocket-pool", "arbitrum", "0xB766039cc6DB368759C1E56B79AFfE831d0Cc507", 18, asset=True)

In [142]:
s["debt"]["btc"]["decimals"] = 8

In [87]:
s["debt"] = {}
# s["asset"]["usdc"] = {}
# s["debt"]["optimism"] = {}
# s["debt"]["polygon"] = {}

In [ ]:
s["asset"]["usdc"] = {}

In [69]:
# s["debt"] = {}
# s["debt"]["arbitrum"] = {}
# s["debt"]["optimism"] = {}
# s["debt"]["polygon"] = {}

In [70]:
s["debt"]["polygon"]["btc"] = "0x92b42c66840c7ad907b4bf74879ff3ef7c529473"

In [66]:
p.save_portfolio(s)

In [92]:
p.wallet_balance(s["chains"]["polygon"], s["wallets"]["main"], "0xe50fa9b3c56ffb159cb0fca61f5c9d750e8128c8", 18)

2.363816466036041

In [124]:
def generate_prices(s):
    prices = set([])
    for k in s['asset']:
        prices.add(s['asset'][k]['coingecko'])
    for k in s['debt']:
        prices.add(s['debt'][k]['coingecko'])
    prices = dict.fromkeys(prices, 0)
    for k in prices:
        prices[k] = coingecko_price(k)
    return prices

In [ ]:
p.wallet_balance(s["chains"]["polygon"], s["wallets"]["main"], "0xe50fa9b3c56ffb159cb0fca61f5c9d750e8128c8", 18)


In [7]:
def generate_amounts(s, wallet):
    amounts = {"asset":{"eth":0}, "debt":{}}
    for t in ['asset', 'debt']:
        for k in s[t].keys():
            a = s[t][k]
            amounts[t][k] = 0
            for c in a.keys():
                if c not in s["chains"].keys(): continue
                amounts[t][k] += p.wallet_balance(s['chains'][c], wallet, a[c], a["decimals"])
    for c in ["optimism", "arbitrum"]:
        amounts["asset"]["eth"] += p.eth_balance(s["chains"][c], wallet)
    return amounts
            
    

In [70]:
a = generate_amounts(s, s["wallets"]["main"])

In [71]:
a

{'asset': {'eth': 0.9962923862731996,
  'usdc': 0.0,
  'aweth': 2.363829489993154,
  'rpl': 26.210720631654},
 'debt': {'btc': 0.07241783}}

In [125]:
def coingecko_price(name):
    res = requests.get(f"https://api.coingecko.com/api/v3/coins/{name}?tickers=false&community_data=false&developer_data=false").json()
    return float(res["market_data"]["current_price"]["usd"])

In [67]:
prices = p.generate_prices(s)

In [68]:
prices

{'bitcoin': 19797.48,
 'rocket-pool': 23.71,
 'ethereum': 1555.72,
 'usd-coin': 1.001}

In [69]:
def calculate_position(s, a, p):
    for t in ['asset', 'debt']:
        for k,v in a[t].items():
            a[t][k] = v*p[s[t][k]['coingecko']]

In [72]:
pos = calculate_position(s, a, prices)

In [73]:
a

{'asset': {'eth': 1549.9519911729421,
  'usdc': 0.0,
  'aweth': 3677.4568141721497,
  'rpl': 621.4561861765163},
 'debt': {'btc': 1433.6905410684}}

In [76]:
class colors:
    PURPLE = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [82]:
print(colors.BOLD + "test asdfadsf asdfaASDFASDF" + colors.ENDC)

test asdfadsf asdfaASDFASDF


In [229]:
{**a['asset'], **a['debt']}.keys()

dict_keys(['eth', 'usdc', 'aweth', 'rpl', 'btc'])

In [197]:
def print_main_portfolio(s, a, eth):
    total = 0
    for k in a['asset']:
        total += a['asset'][k]
    print_statements = []
    print_statements.append(colors.BOLD + "MAIN" + colors.ENDC)

    max_len_token = max(list(map(lambda x: len(x), (a['asset'] | a['debt']).keys())))
    print_statements.append(colors.UNDERLINE + "ASSETS" + colors.ENDC)
    for k in a['asset']:
        if a['asset'][k] == 0: continue
        text = f"{k}: {(max_len_token+1-len(k))*' '}{colors.GREEN}{round(a['asset'][k]/eth, 3)}Ξ{colors.ENDC}"
        text = f"{text}{' '*(28-len(text))}{int(a['asset'][k]*100/total)}%"
        print_statements.append(text)
    if len(a['debt']) > 0: print_statements.append(colors.UNDERLINE + "DEBT" + colors.ENDC)
    for k in a['debt']:
        if a['debt'][k] == 0: continue
        text = f"{k}: {(max_len_token-len(k))*' '}{colors.RED}-{round(a['debt'][k]/eth, 3)}Ξ{colors.ENDC}"
        print_statements.append(text)
        
    for k in a['debt']:
        total -= a['debt'][k]
    print_statements.append(f"TOTAL: {round(total/eth, 3)}Ξ")
    return print_statements

In [198]:
def print_trade_portfolio(s, trade, play, eth):
    
    total = 0
    for k in trade['asset']:
        total += trade['asset'][k]
    for k in play['asset']:
        total += play['asset'][k]
        
    print_statements = []
        
    print_statements.append(colors.BOLD + "FUN" + colors.ENDC)
    for k in play['asset']:
        if play['asset'][k] == 0: continue
        text = f"{k}: {(5-len(k))*' '}{colors.GREEN}{round(play['asset'][k]/eth, 3)}Ξ{colors.ENDC}"
        text = f"{text}{' '*(28-len(text))}{int(play['asset'][k]*100/total)}%"
        print_statements.append(text)
    print_statements.append(colors.BOLD + "TRADE" + colors.ENDC)
    for k in trade['asset']:
        if trade['asset'][k] == 0: continue
        text = f"{k}: {(5-len(k))*' '}{colors.GREEN}{round(trade['asset'][k]/eth, 3)}Ξ{colors.ENDC}"
        text = f"{text}{' '*(28-len(text))}{int(trade['asset'][k]*100/total)}%"
        print_statements.append(text)
    print_statements.append(f"TOTAL: {round(total/eth, 3)}Ξ")
    return print_statements

In [117]:
trade = generate_amounts(s, s["wallets"]["trade"])
play = generate_amounts(s, s["wallets"]["fun"])
calculate_position(s, trade, prices)
calculate_position(s, play, prices)

In [199]:
ps = print_trade_portfolio(s, trade, play, prices['ethereum'])

In [200]:
for i in ps:
    print(i)

FUN
eth:   0.096Ξ      66%
usdc:  0.016Ξ      10%
TRADE
eth:   0.034Ξ      23%
TOTAL: 0.145Ξ


In [201]:
psm = print_main_portfolio(s, a, prices['ethereum'])

In [202]:
for i in psm:
    print(i)

MAIN
ASSETS
eth:    0.996Ξ     26%
aweth:  2.364Ξ     62%
rpl:    0.399Ξ     10%
DEBT
btc:   -0.922Ξ
TOTAL: 2.838Ξ


In [221]:
def print_main_vs_trade(psm, pst):
    left_spacing = max(list(map(lambda x: len(x), psm)))-1
    last = f"{psm[-1]}{(left_spacing-9 - len(psm[-1]))*' '}{'  ≥  ' if (float(psm[-1][6:-1]) >= float(pst[-1][6:-1])) else '  <  '} {pst[-1]}"
    highlight = colors.BLUE if (float(psm[-1][6:-1]) >= float(pst[-1][6:-1])) else colors.RED 
    last = highlight + last + colors.ENDC
    psm = psm[:-1]
    pst = pst[:-1]
    height = max(len(psm), len(pst))

    for i in range(height):
        text = ""
        if i < len(psm):
            text += psm[i] + " "*(left_spacing - len(psm[i]))
            text += "     "
        else:
            text += " "*left_spacing
        if i < len(pst):
            text += pst[i]
        print(text)
    print(last)
    
            
        

In [222]:
print_main_vs_trade(psm, ps)

MAIN                       FUN
ASSETS                     eth:   0.096Ξ      66%
eth:    0.996Ξ     26%     usdc:  0.016Ξ      10%
aweth:  2.364Ξ     62%     TRADE
rpl:    0.399Ξ     10%     eth:   0.034Ξ      23%
DEBT                       
btc:   -0.922Ξ            
TOTAL: 2.838Ξ          ≥   TOTAL: 0.145Ξ


In [204]:
psm[-1]

'TOTAL: 2.838Ξ'

In [31]:
max(list(map(lambda x: len(x), a['asset'].keys())))

5